In [2]:
import os
from connection_complexity.data.raw_data.EDF.edf_helpers import read_edf
import connection_complexity.data.rereference_helpers as reref_helpers
from tqdm import tqdm

from mne import make_fixed_length_epochs
import pandas as pd
import numpy as np

In [ ]:
reref_helpers.rereference()

NameError: name 'reref_helpers' is not defined

In [ ]:
project_path = "/home/dan/data/connectivity/pyspi_testing"

In [14]:
pids = [106,95,39] # all 2048 hz pts
npy_output_path = f"/media/dan/Data/git/ubiquitous-spork/bipolar_reref/random_three_pid/run_data/npy"
contacts_used_output_path = f"/media/dan/Data/git/ubiquitous-spork/bipolar_reref/random_three_pid/run_data/contacts_used"
# make output paths
if not os.path.exists(npy_output_path):
    os.makedirs(npy_output_path)

if not os.path.exists(contacts_used_output_path):
    os.makedirs(contacts_used_output_path)

for pid in pids:
    file = f"/media/dan/Data/data/baseline_patients/baseline_edfs/{pid:03}_Baseline.EDF"
    print(file)
    raw = read_edf(file, preload=True)

    # filter
    sfreq = raw.info["sfreq"]
    nyq_freq = raw.info["sfreq"] // 2 - 1
    line_freq = 60
    l_freq = 0.5
    h_freq = min(nyq_freq, 300)
    raw = raw.filter(l_freq=l_freq, h_freq=h_freq)
    freqs = np.arange(line_freq, max(h_freq, nyq_freq), line_freq)
    raw = raw.notch_filter(freqs=freqs, method="fir")

    # bipolar reference
    raw = reref_helpers.rereference(raw, rereference_scheme="standard")

    contacts_used = raw.ch_names
    # save csv of contacts used
    pd.DataFrame(contacts_used, columns=["contact"]).to_csv(os.path.join(contacts_used_output_path, f"{pid:03}_contacts_used.csv"), index=False)
    
    # non overlapping epochs
    dur_sec = .5
    overlap_sec = 0
    epochs = make_fixed_length_epochs(raw, duration=dur_sec,overlap=overlap_sec, preload=True)
    epochs = epochs.get_data()


    for i, e in enumerate(tqdm(epochs)): 
        start_idx = int(sfreq * dur_sec) * i
        end_idx = start_idx + int(sfreq * dur_sec)
        np.save(os.path.join(npy_output_path, f"{pid:03}_epoch_{start_idx:010d}-{end_idx:010d}.npy"), e)


/media/dan/Data/data/baseline_patients/baseline_edfs/106_Baseline.EDF


100%|██████████| 601/601 [00:00<00:00, 1166.34it/s]


/media/dan/Data/data/baseline_patients/baseline_edfs/095_Baseline.EDF


100%|██████████| 603/603 [00:00<00:00, 1220.85it/s]


/media/dan/Data/data/baseline_patients/baseline_edfs/039_Baseline.EDF


100%|██████████| 605/605 [00:00<00:00, 1276.17it/s]


In [15]:
# define the YAML file location
yaml_file = os.path.join(npy_output_path,"run_config.yaml")

# remove the file if it already exists
if os.path.exists(yaml_file):
    os.remove(yaml_file)

# ps -> rows = processes; columns = time pts.
dim_order = "ps"


import glob
files = list(sorted(list(glob.glob(os.path.join(npy_output_path, "*.npy")))))

for f in sorted(files): # start 1 minute in only use first 5 seconds
    # get the epoch number
    i = os.path.basename(f).split("_")[-1].split(".")[0]

    pid = os.path.basename(f).split("_")[0]
    # define template string and fill in variables
    
    lbl = f"{pid}_epoch_{i}"
    path = os.path.join(npy_output_path, lbl)
    yaml_string = "{{file: {file}.npy, name: {key}, dim_order: {dim_order}, labels: [{key}]}}\n"
    yaml_string_formatted = f"- {yaml_string.format(file=path, key=lbl, dim_order=dim_order)}"

    # append line to file
    with open(yaml_file, "a") as f:
        f.write(yaml_string_formatted)

# make sure yaml loads

In [18]:
import yaml
with open(yaml_file) as d:
    yf = yaml.load(d,Loader=yaml.FullLoader)